# Robotic Arm



A [Robotic arm](https://en.wikipedia.org/wiki/Robotic_arm) is a type of mechanical arm, usually programmable, with similar characteristics of a human arm. The purpose of the robot is to replace the human in difficult and harmful tasks e.g., Industrial production, processing and manufacturing roles. Any task in which extremely precise, fast and repeatable movements are required as well as to automate the process of doing the daily task that humans typically do.

The [arm](https://www.intel.com/content/www/us/en/robotics/robotic-arm.html#:~:text=Most%20industrial%20robotic%20arms%20use,power%20systems%2C%20and%20software%20components) 
contains:
- __Motors and gears__ that allow arm to rotate/move in different directions.
- __Joints__ that consists of several joints.
- __Links__ are the rigid segments that connect the joints.
- __End-effector__ is the device that is attached to the arm's last link e.g., grippers to pick up items.
- __Sensors__, such as cameras, force sensors, or proximity sensors, which allow the arm to sense and respond to its environment.
- __Control system__ is the brain of the robotic arm, which directs the movement and operation of the arm. It is commonly run in a computer, software, and control electronics.

![Model](https://www.mathworks.com/help/examples/fuzzy/win64/invkine_codepad_01.png)

# Problem 1: Accuracy

The accuracy of a robot arm in terms of position and orientation refers to how precisely the arm can move to a specific location and how accurately it can orient its end-effector in a particular direction. This accuracy is typically measured repeatedly along with its positional accuracy.

The accuracy issue is based on the object's displacement from the target area after a series of repeated movements by the arm. It could be error in accuracy along the X or Y-axis due to sensor errors, design, mechanical wear and tear, control algorithms etc.

We can measure the error by:
- [Euclidean system](https://www.kth.se/polopoly_fs/1.1078075.1622113235!/gr22rapport2021-02.pdf) by comparing the __position__ and __orientation__ of the end-effector to the target position and orientation to see the _displacement_ in both __x & y__ directions.

Calculation for accuracy in Eucledian system can be shown like this:
$$ AP_p = {\sqrt{(\bar{x}-x_c)^2+(\bar{y}-y_c)^2}} $$

In [ ]:
#include <iostream>
#include <cmath>

using namespace std;

// Function to calculate the Euclidean distance between two points
double euclidean_distance(double x1, double y1, double z1, double x2, double y2, double z2) {
    return sqrt(pow(x1 - x2, 2) + pow(y1 - y2, 2) + pow(z1 - z2, 2));
}

int main() {
    // Define the target point in 3D space
    double target_x = 10.0;
    double target_y = 5.0;
    double target_z = 7.0;
    
    // Move the robot arm to the target point and record the measured point
    double measured_x = 9.8;
    double measured_y = 4.9;
    double measured_z = 7.2;
    
    // Calculate the Euclidean distance between the target point and the measured point
    double distance = euclidean_distance(target_x, target_y, target_z, measured_x, measured_y, measured_z);
    
    // Output the distance as a measure of the accuracy of the robot arm
    cout << "Distance: " << distance << endl;
    
    return 0;
}


SyntaxError: invalid decimal literal (3741446916.py, line 12)